<a href="https://colab.research.google.com/github/jaejunchoe/HAIDS-Lab/blob/main/Upload_ver03_DeepCoNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import time
import argparse
import inspect
import pandas as pd
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

# ================= Config =================
class Config:
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    train_epochs = 20               # 학습 반복 수
    batch_size = 128                # 배치 사이즈
    learning_rate = 0.002           # 학습률 -> 가중치 업데이트 속도
    l2_regularization = 1e-6        # L2 정규화 -> 과적합 방지 용도 / 과적합이 심하면 값을 늘리는 방식으로 수정하길 권함
    learning_rate_decay = 0.99      # 학습률 감소 비율 -> 점진적으로 줄이면서 안정적인 학습으로 유도하기 위함

    word2vec_file = '/content/drive/MyDrive/IDS/amaxon reviews 2023/glove.6B.50d.txt'

    model_file = '/content/drive/MyDrive/IDS/amaxon reviews 2023/DeepCONN/model_ver02/best_model.pt'
    train_file = '/content/drive/MyDrive/IDS/amaxon reviews 2023/dataset/cleaned_Transnet_T2_train.csv'
    valid_file = '/content/drive/MyDrive/IDS/amaxon reviews 2023/dataset/cleaned_Transnet_T2__valid.csv'
    test_file = '/content/drive/MyDrive/IDS/amaxon reviews 2023/dataset/cleaned_Transnet_T2_test.csv'
    review_count = 10               # 최대 리뷰 수 -> 아이템 당 최대 10개의 리뷰 데이터 활용하겠다
    review_length = 40              # 최대 40단어로 제한하겠다
    lowest_review_count = 0
    PAD_WORD = '<UNK>'              # 패딩 단어

    kernel_count = 100              # CNN에서 필터 개수
    kernel_size = 3                 # 필터 크기
    dropout_prob = 0.5              # 드롭아웃의 확률: 과적합 방지를 위한 비활성화 확률
    cnn_out_dim = 50                # CNN 출력 차원

# ================= Utils =================

In [ ]:
import pandas as pd

In [ ]:

def date(f='%Y-%m-%d %H:%M:%S'):
    return time.strftime(f, time.localtime())

# glove.6B.50d.txt 파일 load하고 embedding하는 함수
def load_embedding(word2vec_file):
    with open(word2vec_file, encoding='utf-8') as f:
        word_emb = [[0]]
        word_dict = {'<UNK>': 0}
        for line in f.readlines():
            tokens = line.split(' ')
            word_emb.append([float(i) for i in tokens[1:]])
            word_dict[tokens[0]] = len(word_dict)
        word_emb[0] = [0] * len(word_emb[1])
    return word_emb, word_dict


# MSE 계산하는 함수 -> train과 valid에서 사용
def predict_mse(model, dataloader, device):
    mse, sample_count = 0, 0
    with torch.no_grad():
        for batch in dataloader:
            user_reviews, item_reviews, ratings = map(lambda x: x.to(device), batch)
            predict = model(user_reviews, item_reviews)
            mse += F.mse_loss(predict, ratings, reduction='sum').item()
            sample_count += len(ratings)
    return mse / sample_count

# Pytorch의 Dataset 클래스를 확장해서 DeepCoNN 모델의 학습에 적합한 데이터셋을 생성하는 Class
class DeepCoNNDataset(Dataset):

    ## csv 파일을 읽고 모델 입력 형식에 맞게 바꾸는 함수
    def __init__(self, data_path, word_dict, config, retain_rui=True):
        self.word_dict = word_dict
        self.config = config
        self.retain_rui = retain_rui                        # 리뷰 포함에 대한 조건
        self.PAD_WORD_idx = self.word_dict[config.PAD_WORD]
        self.review_length = config.review_length
        self.review_count = config.review_count
        self.lowest_r_count = config.lowest_review_count

        df = pd.read_csv(data_path, header=None, names=['userID', 'itemID', 'review', 'rating'])
        df['review'] = df['review'].apply(self._review2id)
        self.sparse_idx = set()

        user_reviews = self._get_reviews(df)
        item_reviews = self._get_reviews(df, 'itemID', 'userID')
        rating = torch.Tensor(df['rating'].to_list()).view(-1, 1)

        # 희소 데이터(self.sparse_idx)를 제외하고 필터 -> lowest_review_count = 0이기에 손실되는 데이터 없을 것
        self.user_reviews = user_reviews[[idx for idx in range(user_reviews.shape[0]) if idx not in self.sparse_idx]]
        self.item_reviews = item_reviews[[idx for idx in range(item_reviews.shape[0]) if idx not in self.sparse_idx]]
        self.rating = rating[[idx for idx in range(rating.shape[0]) if idx not in self.sparse_idx]]


    ## 데이터셋에서 특정 idx로 데이터를 반환함 -> (user_reviews, item_reviews, rating)
    def __getitem__(self, idx):
        return self.user_reviews[idx], self.item_reviews[idx], self.rating[idx]


    ## 데이터셋 전체 길이 반환
    def __len__(self):
        return self.rating.shape[0]


    ## 사용자 또는 아이템 단위로 리뷰 데이터를 그룹화하고, 최대 리뷰 개수(10개)에 따라 정리
    def _get_reviews(self, df, lead='userID', costar='itemID'):
        reviews_by_lead = dict(list(df[[costar, 'review']].groupby(df[lead])))
        lead_reviews = []
        for idx, (lead_id, costar_id) in enumerate(zip(df[lead], df[costar])):
            df_data = reviews_by_lead[lead_id]


            # self.retain_rui = True: 사용자가 작성한 모든 리뷰를 가져와
            # true이기에 else가 실행되지않을 것 -> reviews = df_data['review'].to_list()가 이미 모든 리뷰를 포함하기 때문
            reviews = df_data['review'].to_list() if self.retain_rui else df_data['review'][df_data[costar] != costar_id].to_list()


            if len(reviews) < self.lowest_r_count:
                self.sparse_idx.add(idx)
            reviews = self._adjust_review_list(reviews, self.review_length, self.review_count)      # 개수와 길이 조정
            lead_reviews.append(reviews)
        return torch.LongTensor(lead_reviews)


    ## 리뷰 데이터를 고정된 리뷰 수(review_count)와 리뷰 길이(review_length)로 조정
    def _adjust_review_list(self, reviews, r_length, r_count):
        reviews = reviews[:r_count] + [[self.PAD_WORD_idx] * r_length] * (r_count - len(reviews))
        reviews = [r[:r_length] + [0] * (r_length - len(r)) for r in reviews]
        return reviews

    ## 리뷰 문자열을 단어 임베딩 인덱스 리스트로 변환
    def _review2id(self, review):
        if not isinstance(review, str):
            return []
        return [self.word_dict.get(word, self.PAD_WORD_idx) for word in review.split()]

# ================= Model =================

In [ ]:
# User와 item 리뷰에서 feature 추출하는 역할
class CNN(nn.Module):
    def __init__(self, config, word_dim):
        super(CNN, self).__init__()
        self.kernel_count = config.kernel_count
        self.review_count = config.review_count

        self.conv = nn.Sequential(
            nn.Conv1d(in_channels=word_dim, out_channels=config.kernel_count, kernel_size=config.kernel_size, padding=(config.kernel_size - 1) // 2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(1, config.review_length)),
            nn.Dropout(p=config.dropout_prob))

        self.linear = nn.Sequential(
            nn.Linear(config.kernel_count * config.review_count, config.cnn_out_dim),
            nn.ReLU(),
            nn.Dropout(p=config.dropout_prob))

    # latent 생성하는 함수
    def forward(self, vec):
        latent = self.conv(vec.permute(0, 2, 1))
        latent = self.linear(latent.reshape(-1, self.kernel_count * self.review_count))
        return latent

# 추출된 feature로 상호작용(interaction)해서 최종 예측 rating을 생성
class FactorizationMachine(nn.Module):
    def __init__(self, p, k):                               # latent matrix / p = 입력 벡터의 차원, k = 잠재 차원
        super().__init__()
        self.v = nn.Parameter(torch.rand(p, k) / 10)        # 초기화를 해당 식으로 할당
        self.linear = nn.Linear(p, 1, bias=True)            # 1 = 출력 크기
        self.dropout = nn.Dropout(0.5)

    # latent 생성하는 함수
    def forward(self, x):
        linear_part = self.linear(x)                        # 선형관계 모델링
        inter_part1 = torch.mm(x, self.v) ** 2              # 모든 특징간의 합성곱 -> 입력 데이터 x와 latent matrix(self.v)
        inter_part2 = torch.mm(x ** 2, self.v ** 2)         # 개별 특징의 제곱으로 행렬 곱 전개 -> x^2와 v^2

        # inter_part1은 전체 효과 / inter_part2는 개별 효과의 합산 -> 이 둘을 빼면? = 전체 효과에서 개별 효과를 뺀 값
        # 여기에서 뺀 값의 의미 = 개별 효과로 설명되지 않는 부분 -> 상호작용 효과
        pair_interactions = torch.sum(inter_part1 - inter_part2, dim=1, keepdim=True)       # 값의 차이로 상호작용 추출 + 값 변환
        pair_interactions = self.dropout(pair_interactions)
        return linear_part + 0.5 * pair_interactions




# 전체 모델 구조 정의 + CNN과 FactorizationMachine(FM) 결합해서 예측
class DeepCoNN(nn.Module):
    def __init__(self, config, word_emb):
        super(DeepCoNN, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(torch.Tensor(word_emb))
        self.cnn_u = CNN(config, word_dim=self.embedding.embedding_dim)             # user 리뷰 처리
        self.cnn_i = CNN(config, word_dim=self.embedding.embedding_dim)             # item 리뷰 처리
        self.fm = FactorizationMachine(config.cnn_out_dim * 2, 10)                  # latent로 최종 rating 예측

    # 사용자 리뷰(user_review)와 아이템 리뷰(item_review)를 입력받아 처리하는 함수
    def forward(self, user_review, item_review):

        # 리뷰 데이터를 배치(batch) 단위로 평탄화(reshape)하여 임베딩에 적합하도록 변환
        new_batch_size = user_review.shape[0] * user_review.shape[1]
        user_review = user_review.reshape(new_batch_size, -1)
        item_review = item_review.reshape(new_batch_size, -1)

        # 임베딩 벡터 변환
        u_vec = self.embedding(user_review)
        i_vec = self.embedding(item_review)

        user_latent = self.cnn_u(u_vec)
        item_latent = self.cnn_i(i_vec)

        concat_latent = torch.cat((user_latent, item_latent), dim=1)        # latent 결합
        return self.fm(concat_latent)



# ================= Training =================

In [ ]:
# ================= Training =================
def train(train_dataloader, valid_dataloader, model, config, model_path):
    print(f'{date()}## Start the training!')
    train_mse = predict_mse(model, train_dataloader, config.device)
    valid_mse = predict_mse(model, valid_dataloader, config.device)
    print(f'{date()}#### Initial train mse {train_mse:.6f}, validation mse {valid_mse:.6f}')
    start_time = time.perf_counter()

    # weight_decay = L2 정규화
    opt = torch.optim.Adam(model.parameters(), config.learning_rate, weight_decay=config.l2_regularization)

    # 학습률 스케줄
    # ExponentialLR을 사용하여 학습률을 매 에포크마다 learning_rate_decay 비율로 줄이는 방식
    lr_sch = torch.optim.lr_scheduler.ExponentialLR(opt, config.learning_rate_decay)

    best_loss = float('inf')    # 최소값을 위한 초기값 설정 -> validation에서의 최소 mse값

    for epoch in range(config.train_epochs):
        model.train()
        total_loss, total_samples = 0, 0
        for batch in train_dataloader:
            user_reviews, item_reviews, ratings = map(lambda x: x.to(config.device), batch)
            predict = model(user_reviews, item_reviews)

            # F.mse_loss를 사용해 예측값과 실제값 간의 평균 제곱 오차(MSE)를 계산
            loss = F.mse_loss(predict, ratings, reduction='sum')
            opt.zero_grad()
            loss.backward()
            opt.step()

            total_loss += loss.item()
            total_samples += len(ratings)

        lr_sch.step()
        model.eval()
        valid_mse = predict_mse(model, valid_dataloader, config.device)
        train_loss = total_loss / total_samples
        print(f"{date()}#### Epoch {epoch:3d}; train mse {train_loss:.6f}; validation mse {valid_mse:.6f}")

        if best_loss > valid_mse:
            best_loss = valid_mse
            torch.save(model, model_path)

    print(f'{date()}## End of training! Time used {time.perf_counter() - start_time:.0f} seconds.')






# ================= Test, Main =================

In [ ]:
import torch


# ================= Testing =================
def test(dataloader, model, config):
    print(f'{date()}## Start the testing!')
    test_loss = predict_mse(model, dataloader, config.device)
    print(f"{date()}## Test end, test mse is {test_loss:.6f}")


# ================= Main =================
if __name__ == '__main__':
    config = Config()
    print(config)
    print(f'{date()}## Load embedding and data...')
    word_emb, word_dict = load_embedding(config.word2vec_file)

    train_dataset = DeepCoNNDataset(config.train_file, word_dict, config)
    valid_dataset = DeepCoNNDataset(config.valid_file, word_dict, config, retain_rui=False)
    test_dataset = DeepCoNNDataset(config.test_file, word_dict, config, retain_rui=False)
    train_dataloader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
    valid_dataloader = DataLoader(valid_dataset, batch_size=config.batch_size)
    test_dataloader = DataLoader(test_dataset, batch_size=config.batch_size)

    # 모델 생성 및 학습
    model = DeepCoNN(config, word_emb).to(config.device)
    os.makedirs(os.path.dirname(config.model_file), exist_ok=True)
    train(train_dataloader, valid_dataloader, model, config, config.model_file)

    # 로드할 때 필요한 클래스들을 안전한 글로벌에 추가

    torch.serialization.add_safe_globals([
        DeepCoNN,
        CNN,
        FactorizationMachine,
        torch.nn.modules.sparse.Embedding,
        torch.nn.modules.container.Sequential,
        torch.nn.modules.conv.Conv1d,
        torch.nn.modules.activation.ReLU,
        torch.nn.modules.pooling.MaxPool2d,
        torch.nn.modules.dropout.Dropout,
        torch.nn.modules.linear.Linear
    ])
    loaded_model = torch.load(config.model_file)

    test(test_dataloader, loaded_model, config)

    print(f"Train dataset size: {len(train_dataset)}")
    print(f"Valid dataset size: {len(valid_dataset)}")
    print(f"Test dataset size: {len(test_dataset)}")

    # 데이터 확인
    for i, (user_review, item_review, rating) in enumerate(train_dataloader):
        print(f"Batch {i+1}:")
        print(f"User Review Tensor: {user_review.shape}")
        print(f"Item Review Tensor: {item_review.shape}")
        print(f"Rating Tensor: {rating.shape}")
        break

2025-03-29 09:55:14## Load embedding and data...
2025-03-29 09:55:27## Start the training!
2025-03-29 09:55:27#### Initial train mse 11.684132, validation mse 11.840652
2025-03-29 09:55:28#### Epoch   0; train mse 5.387804; validation mse 6.064975
2025-03-29 09:55:28#### Epoch   1; train mse 2.411958; validation mse 3.843110
2025-03-29 09:55:29#### Epoch   2; train mse 2.061943; validation mse 3.581172
2025-03-29 09:55:30#### Epoch   3; train mse 1.954420; validation mse 3.242029
2025-03-29 09:55:31#### Epoch   4; train mse 1.862581; validation mse 3.120563
2025-03-29 09:55:32#### Epoch   5; train mse 1.808903; validation mse 2.913261
2025-03-29 09:55:32#### Epoch   6; train mse 1.749055; validation mse 2.979349
2025-03-29 09:55:33#### Epoch   7; train mse 1.715458; validation mse 3.149965
2025-03-29 09:55:33#### Epoch   8; train mse 1.687228; validation mse 3.096098
2025-03-29 09:55:34#### Epoch   9; train mse 1.641868; validation mse 3.411059
2025-03-29 09:55:35#### Epoch  10; train 